In [1]:
import pandas as pd
import numpy as np

## inpute CONPROP_after

In [2]:
path = "../Data/Online_Data/trainning_data/corporate_attributes/"
outpath = "../Processed_Data/"

In [3]:
df_corp_inv = pd.read_csv(path+'el_company_history_inv.csv')
df_corp_inv.head(2)

<ipython-input-3-dfa24790bba8>:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  df_corp_inv = pd.read_excel(path+'el_company_history_inv.xlsx')


,entid,entid_inv,CONAM_before,CONAM_after,CONPROP_before,CONPROP_after,ttype,ATDATE
0,122697049,3515,8.103057e+06,NaN,100.439318,NaN,4.0,NaT
1,122697049,3515,8.103055e+06,8.103056e+06,99.930037,NaN,2.0,NaT


In [4]:
df_corp_inv['CONPROP_after'] = df_corp_inv['CONPROP_after'].fillna(
    df_corp_inv['CONAM_before']/df_corp_inv['CONAM_after']*df_corp_inv['CONPROP_before'])


In [5]:
df_corp_inv.head(2)

,entid,entid_inv,CONAM_before,CONAM_after,CONPROP_before,CONPROP_after,ttype,ATDATE
0,122697049,3515,8.103057e+06,NaN,100.439318,NaN,4.0,NaT
1,122697049,3515,8.103055e+06,8.103056e+06,99.930037,99.930031,2.0,NaT


## create 'large SH'

In [6]:
df_corp_inv['big'] = 0

def procee_big(row):
    if str(row['CONPROP_after']) == "nan": 
        row['big'] = 2
    elif row['CONPROP_after'] >= 50:
        row['big'] = 1
    return row
df_corp_inv = df_corp_inv.apply(procee_big,  axis = 1)

In [7]:
df_corp_inv['big'].value_counts()

0    5438
1    2881
2    1851
Name: big, dtype: int64

## create new variables

In [8]:
df_corp_inv['net_change'] = df_corp_inv['CONPROP_after'] - df_corp_inv['CONPROP_before']

In [9]:
df_corp_inv_unique = df_corp_inv[["entid", "entid_inv"]].groupby("entid")
df_corp_inv_unique = df_corp_inv_unique.nunique()
# df_corp_inv_unique = df_corp_inv_unique.drop(columns=['entid'])
df_corp_inv_unique = df_corp_inv_unique.reset_index()

In [10]:
df_corp_inv_unique.head()

,entid,entid_inv
0,109606,2
1,243165,2
2,683429,1
3,703531,1
4,2118849,1


In [11]:
df_corp_inv_unique.columns = ['entid', 'entid_inv_Uniq']

In [12]:
df_corp_inv = pd.merge(df_corp_inv, df_corp_inv_unique, on = 'entid')
df_corp_inv.head()

,entid,entid_inv,CONAM_before,CONAM_after,CONPROP_before,CONPROP_after,ttype,ATDATE,big,net_change,entid_inv_Uniq
0,122697049,3515,8.103057e+06,NaN,100.439318,NaN,4.0,NaT,2,NaN,3
1,122697049,3515,8.103055e+06,8.103056e+06,99.930037,99.930031,2.0,NaT,1,-0.000005,3
2,122697049,3515,8.103060e+06,8.103058e+06,99.042763,99.042782,2.0,NaT,1,0.000019,3
3,122697049,39640120,5.584795e+06,NaN,68.778979,NaN,4.0,NaT,2,NaN,3
4,122697049,0,5.584793e+06,5.584792e+06,69.373550,69.373552,NaN,NaT,1,0.000002,3


In [13]:
df_corp_inv['vol'] = (df_corp_inv['CONAM_before'] + df_corp_inv['CONAM_after'])/df_corp_inv['entid_inv_Uniq']

In [14]:
df_corp_inv = df_corp_inv.drop(['ATDATE'], axis=1)

In [15]:
df_corp_inv.ttype.value_counts()

2.0    4575
4.0    2695
3.0    1488
1.0     365
Name: ttype, dtype: int64

In [16]:
# change data type
df_corp_inv['ttype'] = df_corp_inv['ttype'].astype('category', copy=False)
    
# only dummy seletive categorical variable
corp_dum = pd.get_dummies(df_corp_inv['ttype'])
corp_dum.head(2)

,1.0,2.0,3.0,4.0
0,0,0,0,1
1,0,1,0,0


In [17]:
df_corp_inv_dum = pd.concat([df_corp_inv,corp_dum], axis=1)
for i in ['ttype']:
    del df_corp_inv_dum[i]
    
df_corp_inv_dum.head(2)

,entid,entid_inv,CONAM_before,CONAM_after,CONPROP_before,CONPROP_after,big,net_change,entid_inv_Uniq,vol,1.0,2.0,3.0,4.0
0,122697049,3515,8.103057e+06,NaN,100.439318,NaN,2,NaN,3,NaN,0,0,0,1
1,122697049,3515,8.103055e+06,8.103056e+06,99.930037,99.930031,1,-0.000005,3,5.402037e+06,0,1,0,0


In [18]:
df_corp_inv_dum_g = df_corp_inv_dum[['entid',1.0,2.0,3.0,4.0]].groupby('entid').sum()

In [19]:
df_corp_inv_dum_g = df_corp_inv_dum_g.reset_index()
df_corp_inv_dum_g.head()

,entid,1.0,2.0,3.0,4.0
0,109606,0.0,3.0,0.0,0.0
1,243165,0.0,3.0,0.0,0.0
2,683429,0.0,0.0,0.0,2.0
3,703531,0.0,4.0,0.0,0.0
4,2118849,0.0,2.0,0.0,0.0


In [20]:
df_corp_inv.columns

Index(['entid', 'entid_inv', 'CONAM_before', 'CONAM_after', 'CONPROP_before',
       'CONPROP_after', 'ttype', 'big', 'net_change', 'entid_inv_Uniq', 'vol'],
      dtype='object')

In [21]:
numerical_columns = ['CONAM_before', 'CONAM_after', 'CONPROP_before',
       'CONPROP_after', 'net_change', 'entid_inv_Uniq', 'vol']
def get_numerical_df(df, columns):
    keep_columns = columns + ['entid']
    df_numerical = df[keep_columns].groupby("entid")
    df_numerical = df_numerical.mean()
    df_numerical = df_numerical.reset_index()
    return df_numerical
df_numerical = get_numerical_df(df_corp_inv, numerical_columns)
df_numerical.shape

(3142, 8)

In [22]:
df_numerical.head()

,entid,CONAM_before,CONAM_after,CONPROP_before,CONPROP_after,net_change,entid_inv_Uniq,vol
0,109606,316.104106,317.431291,63.901725,63.635511,-0.266214,2,316.767698
1,243165,39245.320086,39243.633547,65.005836,65.008299,0.002462,2,39244.476816
2,683429,500.469199,NaN,49.919307,NaN,NaN,1,NaN
3,703531,35.480602,36.025610,34.955911,34.121672,-0.834240,1,71.506213
4,2118849,51.026864,51.086143,49.834791,49.857574,0.022783,1,102.113007


In [23]:
df_corp_inv_big = df_corp_inv.groupby('entid')['big'].apply(list)

In [24]:
df_corp_inv_big = pd.DataFrame(df_corp_inv_big)
df_corp_inv_big.reset_index()
# df_corp_inv_big.columns = ['entid', 'big']

,entid,big
0,109606,"[1, 1, 0]"
1,243165,"[1, 0, 1]"
2,683429,"[2, 2]"
3,703531,"[0, 0, 0, 0]"
4,2118849,"[1, 0]"
...,...,...
3137,1231510113,"[1, 0]"
3138,1231792344,[1]
3139,1232004753,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
3140,1232844192,"[0, 1]"


In [25]:
def process_big(x):
    if 1 in x: return 1
    elif 0 in x: return 0
    else: return 2
df_corp_inv_big['big'] = df_corp_inv_big['big'].apply(process_big)

In [26]:
df_corp_inv_big = df_corp_inv_big.reset_index()

,entid,big
0,109606,1
1,243165,1
2,683429,2
3,703531,0
4,2118849,1
...,...,...
3137,1231510113,1
3138,1231792344,1
3139,1232004753,2
3140,1232844192,1


In [27]:
df_corp = pd.merge(df_corp_inv_big, df_corp_inv_dum_g, on = 'entid')

In [28]:
df_co = pd.merge(df_corp, df_numerical, on = 'entid')

In [29]:
df_co.head()

,entid,big,1.0,2.0,3.0,4.0,CONAM_before,CONAM_after,CONPROP_before,CONPROP_after,net_change,entid_inv_Uniq,vol
0,109606,1,0.0,3.0,0.0,0.0,316.104106,317.431291,63.901725,63.635511,-0.266214,2,316.767698
1,243165,1,0.0,3.0,0.0,0.0,39245.320086,39243.633547,65.005836,65.008299,0.002462,2,39244.476816
2,683429,2,0.0,0.0,0.0,2.0,500.469199,NaN,49.919307,NaN,NaN,1,NaN
3,703531,0,0.0,4.0,0.0,0.0,35.480602,36.025610,34.955911,34.121672,-0.834240,1,71.506213
4,2118849,1,0.0,2.0,0.0,0.0,51.026864,51.086143,49.834791,49.857574,0.022783,1,102.113007


In [30]:
df_co.to_csv(outpath+'el_company_history_inv_processed.csv', index=False)